<a href="https://colab.research.google.com/github/nllllibeth/ML/blob/master/Lab_2_ML_Sheina_Elizaveta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данной лабораторной работе показаны основы анализа текстовой информации. В ходе её выполнения мы познакомились с этапами предварительной подготовки данных, а также применили машинное обучение для задачи классификации.

Датасет с текстами песен разных жанров: https://www.kaggle.com/mehedihasan9021/movie-script-dataset




Задание 1

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/dataset.csv", sep = ',')
data.head()


Mounted at /content/drive


,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre     558 non-null    object
 1   lyrics    558 non-null    object
 2   SongInfo  558 non-null    object
dtypes: object(3)
memory usage: 13.2+ KB


In [ ]:
columns = data[['genre', 'lyrics']]

In [ ]:
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


Посмотрим какие жанры присутствуют в датасете. Для этого выведем уникальные значения столбца:

In [ ]:
columns['genre'].unique()

array(['Christian', 'Country', 'Hip-Hop', 'Pop', 'Rock', 'R&B'],
      dtype=object)

Посчитаем также сколько песен каждого жанра представлено:

In [ ]:
columns['genre'].value_counts()

Pop          100
Rock          95
Christian     94
R&B           91
Hip-Hop       91
Country       87
Name: genre, dtype: int64

Оставляем только 2 жанра, которые планируем отличать друг от друга. Для примера будут использованы Christian и Hip-Hop, позднее в формулировке задания нужно будет сгенерировать собственную пару жанров. 

In [ ]:
import random 
lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 


ваши жанры Rock и Country


In [ ]:
columns = columns[(columns.genre == 'Rock') | (columns.genre == "Country")]
columns

,genre,lyrics
94,Country,Your cheatin heart will make you weep Youll c...
95,Country,I was standing by the window On a dark and cl...
96,Country,Blue moon of Kentucky keep on shinin Shine on...
97,Country,Hear that lonesome whippoorwill He sounds too...
98,Country,(Hank Cochran) Make the world go away And ge...
...,...,...
462,Rock,Cant explain all the feelings that youre maki...
463,Rock,"One foot on the brake and one on the gas, hey..."
464,Rock,Carry on my wayward son Therell be peace when...
465,Rock,"Ooh yeah Turn it up Come on Im working hard,..."


###Предварительная обработка текстовых данных

Для того, чтобы применять текстовые данные для обучения той или иной модели машинного обучения, их нужно привести в подходящий для этого вид. В этом нам помогут следующие шаги предварительной подготовки:

####Приведение к нижнему регистру

Так как слова (например) "Beer" и "beer" будут считаться разными словами из-за разницы в регистре буквы b, а первая буква предложения, как правило, является заглавной, важно привести все слова предложения к нижнему регистру. Таким образом, "beer", встретившееся в начале предложения и в его середине, будут распознаны как одно слово.

In [ ]:
lowered = columns['lyrics'].str.lower()

In [ ]:
columns['lowered'] = lowered
lowered.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


94     your cheatin heart will make you weep youll c...
95     i was standing by the window on a dark and cl...
96     blue moon of kentucky keep on shinin shine on...
97     hear that lonesome whippoorwill he sounds too...
98     (hank cochran)  make the world go away and ge...
Name: lyrics, dtype: object

####Токенизация

Имеющиеся тексты нужно разбить на отдельные слова. Такой процесс называется ***токенизация***.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened
tokened.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


94    [your, cheatin, heart, will, make, you, weep, ...
95    [i, was, standing, by, the, window, on, a, dar...
96    [blue, moon, of, kentucky, keep, on, shinin, s...
97    [hear, that, lonesome, whippoorwill, he, sound...
98    [(, hank, cochran, ), make, the, world, go, aw...
dtype: object

####Удаление стоп-слов

***Стоп-словами*** называются распространённые слова, не несущие особой смысловой нагрузки, а значит никак не помогающие в последующей задаче классификации. Такие слова мы считаем шумом, и, соответственно, удаляем.

Существует специальный корпус стоп-слов на разных языках. Так как сейчас мы имеем дело с текстами на английском языке, выведем стоп-слова для английского:

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop
without_stop[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['cheatin, heart, make, weep, youll, cry, cry, try, sleep, sleep, wont, come, whole, night, cheatin, heart, tell, tears, come, like, fallin, rain, youll, toss, around, call, name, youll, walk, floor, way, cheatin, heart, tell, cheatin, heart, find, day, crave, love, threw, away, time, come, youll, blue, cheatin, heart, tell, tears, come, like, fallin, rain, youll, toss, around, call, name, youll, walk, floor, way, cheatin, heart, tell',
 'standing, window, dark, cloudy, day, saw, hearse, come, rolling, carry, mother, away, circle, unbroken, ,, lord, ,, theres, better, home, awaiting, sky, ,, lord, ,, sky, lord, ,, told, undertaker, ,, undertaker, please, drive, slow, body, holding, lord, lord, ,, hate, see, go, circle, unbroken, ,, lord, ,, theres, better, home, awaiting, sky, ,, lord, ,, sky, followed, close, behind, tried, hold, brave, could, hide, sorrow, laid, grave, circle, unbroken, ,, lord, ,, theres, better, home, awaiting, sky, ,, lord, ,, sky, went, home, lord, ,, home, lones

####Лемматизация

У одного и того же слова бывают разные формы. Например, dances - форма слова dance. Лемматизация - это приведение к ***лемме*** - исходной форме слова. Её также необходимо применить к нашим данным, чтобы разные формы слова не считались за отдельные слова.

Существует целый ряд лемматизаторов, которые показывают себя с разной эффективностью на разных языках. Для работы с английским языком эффективен **WordNetLemmatizer**

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns['lemmatized'] = lemma

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Таким образом, на каждом этапе мы добавляли столбец с той или иной модификацией, чтобы получить пригодные для обучения данные. Посмотрим на датафрейм, который получился:

In [ ]:
columns

,genre,lyrics,lowered,tokened,without_stop,lemmatized
94,Country,Your cheatin heart will make you weep Youll c...,your cheatin heart will make you weep youll c...,"[your, cheatin, heart, will, make, you, weep, ...","cheatin, heart, make, weep, youll, cry, cry, t...","cheatin, heart, make, weep, youll, cry, cry, t..."
95,Country,I was standing by the window On a dark and cl...,i was standing by the window on a dark and cl...,"[i, was, standing, by, the, window, on, a, dar...","standing, window, dark, cloudy, day, saw, hear...","standing, window, dark, cloudy, day, saw, hear..."
96,Country,Blue moon of Kentucky keep on shinin Shine on...,blue moon of kentucky keep on shinin shine on...,"[blue, moon, of, kentucky, keep, on, shinin, s...","blue, moon, kentucky, keep, shinin, shine, one...","blue, moon, kentucky, keep, shinin, shine, one..."
97,Country,Hear that lonesome whippoorwill He sounds too...,hear that lonesome whippoorwill he sounds too...,"[hear, that, lonesome, whippoorwill, he, sound...","hear, lonesome, whippoorwill, sounds, blue, fl...","hear, lonesome, whippoorwill, sounds, blue, fl..."
98,Country,(Hank Cochran) Make the world go away And ge...,(hank cochran) make the world go away and ge...,"[(, hank, cochran, ), make, the, world, go, aw...","(, hank, cochran, ), make, world, go, away, ge...","(, hank, cochran, ), make, world, go, away, ge..."
...,...,...,...,...,...,...
462,Rock,Cant explain all the feelings that youre maki...,cant explain all the feelings that youre maki...,"[cant, explain, all, the, feelings, that, your...","cant, explain, feelings, youre, making, feel, ...","cant, explain, feelings, youre, making, feel, ..."
463,Rock,"One foot on the brake and one on the gas, hey...","one foot on the brake and one on the gas, hey...","[one, foot, on, the, brake, and, one, on, the,...","one, foot, brake, one, gas, ,, hey, well, ,, t...","one, foot, brake, one, gas, ,, hey, well, ,, t..."
464,Rock,Carry on my wayward son Therell be peace when...,carry on my wayward son therell be peace when...,"[carry, on, my, wayward, son, therell, be, pea...","carry, wayward, son, therell, peace, done, lay...","carry, wayward, son, therell, peace, done, lay..."
465,Rock,"Ooh yeah Turn it up Come on Im working hard,...","ooh yeah turn it up come on im working hard,...","[ooh, yeah, turn, it, up, come, on, im, workin...","ooh, yeah, turn, come, im, working, hard, ,, y...","ooh, yeah, turn, come, im, working, hard, ,, y..."


####Разделим данные на обучающую и тестовую выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки 

> 



y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre


> 


x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр


> 



y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)

In [ ]:
columns.genre.value_counts()

Rock       95
Country    87
Name: genre, dtype: int64

####Векторизация

Следующий этап - ***векторизация***, то есть представление текста в численном виде, чтобы закодированные данные в дальнейшем использовать на модели.

Иногда помимо отдельных слов для улучшения качества обучения будет полезно использовать также комбинации из 2-3 слов. Здесь мы используем векторизатор CountVectorizer, который имеет встроенный метод - мешок n-грамм.

> n = 1 - униграмма (для обучения используются слова по отдельности)

> n = 2 - биграмма (для обучения используются пары слов)

> n = 3 - триграмма (для обучения используются тройки слов)







In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Задаём для векторизатора ngram_range=(1, 3), то есть используем все варианты n от 1 до 3 и прибегаем как к униграммам, так и к биграммам и триграммам:

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

### Классификация

Для задачи классификации используем Наивный Байесовский Классификатор - простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими предположениями о независимости. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB 

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

In [ ]:
vectorized_x_test = vectorizer.transform(x_test)

Посмотрим предсказания для тестовой выборки

In [ ]:
clf.predict(vectorized_x_test)

array(['Rock'], dtype='<U4')

Получим оценки классификации:

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

     Country       0.76      0.62      0.68        26
        Rock       0.71      0.83      0.76        29

    accuracy                           0.73        55
   macro avg       0.73      0.72      0.72        55
weighted avg       0.73      0.73      0.72        55



Задание 2

In [ ]:
from re import L
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
lyr1 = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Led Zeppelin and Willy Nelson.csv', sep = ',')
columns = lyr1[['lyrics']]
columns.loc[0, 'genre'] = 'Rock'
columns.loc[1, 'genre'] = 'Country'

lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

nltk.download('stopwords')
from nltk.corpus import stopwords
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

vectorised_samples = vectorizer.transform(columns['lemmatized'])

clf.predict(vectorised_samples)
print('Предполагаемый жанр', clf.predict)


Mounted at /content/drive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Предполагаемый жанр <bound method _BaseNB.predict of MultinomialNB()>


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o